In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path

from hloc import extract_features, match_features, visualization
from hloc import triangulation, pairs_from_retrieval, pairs_from_covisibility,colmap_from_nvm
from hloc import localize_sfm
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
from PIL import Image
import matplotlib.pyplot as plt

# Setup

In [ ]:
dataset = Path('datasets/autovision/')
images = dataset / "local_PinholeType_back/images/"
queries = dataset / "local_PinholeType_back/queries/*.txt"
outputs = Path('outputs/one_north_instant/')
# !rm -rf $outputs #clear old files
sfm_pairs = outputs / 'pairs-sfm-db.txt' #top 20 most covisible
loc_pairs = outputs / 'pairs-dir-q.txt' #top 20 retrieved by deep image retrieval
sfm_dir = outputs / 'sfm'
results = outputs / 'results.txt'

retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen'] 
matcher_conf = match_features.confs['superglue-fast']

In [ ]:
references = [str(p.relative_to(dataset)) for p in (images).iterdir()]
print(len(references), "files")

# 提取 database中的特征点（superpoint）& Match the database images（superglue）

这里先对当前目录下已经存在的图片进行特征点提取，默认初始情况应该是db文件夹非空，query为空文件夹

In [ ]:
features = extract_features.main(feature_conf, images, outputs)

这里选出db文件夹中各图像最共视的20个pair，写入pairs-sfm-db.txt

In [ ]:
# select the top 20 most covisible neighbors for each image in the database
pairs_from_covisibility.main(
    outputs / "sfm_sift", sfm_pairs, num_matched=20)

In [ ]:
# match the dabase images
sfm_matches = match_features.main(matcher_conf, sfm_pairs, feature_conf["output"], outputs)

# 根据原sfm模型重建一个稀疏的sfm模型
Triangulate the sparse 3D pointcloud given the matches and the reference poses stored in the SIFT COLMAP model.

In [ ]:
reconstruction = triangulation.main(sfm_dir, outputs / "sfm_sift", images, sfm_pairs, features, sfm_matches)

# 提取global descriptors & 粗检索query images

In [ ]:
global_descriptors = extract_features.main(retrieval_conf, images, outputs)

In [ ]:
features = extract_features.main(feature_conf, images, outputs)
pairs_from_retrieval.main(global_descriptors, loc_pairs, num_matched=20, db_prefix="db", query_prefix="query")

In [ ]:
loc_matches = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

# 定位估计pose
The results saved in results.txt will contain the estimated query poses.

In [ ]:
#localize one query image
localization = localize_sfm.main(
    reconstruction,
    queries,
    loc_pairs,
    features,
    loc_matches,
    results,
    covisibility_clustering=False) 

# Visualization

In [ ]:
visualization.visualize_loc(results, images, reconstruction, images / "db",n=6)

读取results

In [ ]:
r_path = results
names = []
qvecs = []
tvecs = []
with open(r_path, 'r') as f:
    for p in f.read().rstrip('\n').split('\n'):
        if len(p) == 0:
            continue
        data = p.split()
        names.append(data[0])
        qvec = []
        tvec = []
        for i in range(1,5,1):
            qvec.append(data[i])
        qvecs.append(qvec)
        for i in range(5,8,1):
            tvec.append(data[i])
        tvecs.append(tvec)
print(tvecs)

In [ ]:
import pycolmap

fig = viz_3d.init_figure()

for i in range(len(names)):
    q_name = "query/"+names[i]
    camera = pycolmap.infer_camera_from_image(images / q_name)
    pose = pycolmap.Image(name=names[i],tvec=tvecs[i], qvec=qvecs[i])
    viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=names[i])
fig.show()

In [ ]:
# debug 
# 输出检查h5文件
i# Visualizationmport hloc.utils.io
hloc.utils.io.list_h5_names(outputs / "feats-superpoint-n4096-rmax1600.h5")